In [1]:
# Required Libraries
import tensorflow as tf

# Required Libraries
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np

In [2]:
# Load Pre-trained Keras Models
SMR_fh2_model = tf.keras.models.load_model('SMR_fh2.keras')
WE_H2_model = tf.keras.models.load_model('WE_ACM_H2_T_N.keras')

fn2_fair_model = tf.keras.models.load_model('fn2_fair_plpc.keras')
Fn2_P_model = tf.keras.models.load_model('Fn2_P_Fair.keras')

Haber_f_model = tf.keras.models.load_model('Haber_f.keras')
my_model7 = tf.keras.models.load_model('my_model7.h5')

dist_column_f_model = tf.keras.models.load_model('dist_column_f_ffeed_fh2o.keras')
dist_column_x_model = tf.keras.models.load_model('dist_column_xnh3_ffeed_fh2o.keras')

In [3]:
# Load Pre-trained Keras Models for CAPEX
EA_SMR_Model = tf.keras.models.load_model('EA_SMR_Old.keras')

EA_CAS_Model = tf.keras.models.load_model('EA_CAS_Old.keras')
EA_MAS_Model = tf.keras.models.load_model('EA_MAS_Old.keras')

EA_HB_Model = tf.keras.models.load_model('EA_HB_Old.keras')

EA_Dist_Model = tf.keras.models.load_model('EA_Dist_Old.keras')

# Load Pre-trained Keras Models for OPEX
SMR_ne_model = tf.keras.models.load_model('SMR_ne_cal_s.keras')
WE_NE_model = tf.keras.models.load_model('WE_ACM_NE_T_N.keras')

CASU_ne_model = tf.keras.models.load_model('CASU_ne_W.keras')
MASU_ne_model = tf.keras.models.load_model('MASU_ne_kW.keras')

HB_ne_model = tf.keras.models.load_model('HB_ne_cal_s.keras')

dist_ne_model = tf.keras.models.load_model('dist_column_ne_ffeed_fh2o.keras')

In [4]:
# Optimization Model (Pyomo)
model = ConcreteModel()

# Binary decision variables for each route (0 = not selected, 1 = selected)
model.SMR = Var(domain=Binary)
model.WE = Var(domain=Binary)
model.CAS = Var(domain=Binary)
model.MAS = Var(domain=Binary)
model.HB = Var(domain=Binary)
model.MW = Var(domain=Binary)
model.Dist = Var(domain=Binary)
model.Mem = Var(domain=Binary)

# Continuous decision variables
model.FNG = Var(bounds=(80, 120))  # For SMR
model.FH2O = Var(bounds=(255, 375))  # For SMR
model.Tsr = Var(bounds=(900, 1100))  # For SMR
model.Psr = Var(bounds=(29.6, 46.8))  # For SMR
model.T_water = Var(bounds=(50, 80))  # For Water Electrolysis
model.N_water = Var(bounds=(10, 50))  # For Water Electrolysis
model.FH2O_dist = Var(bounds=(100, 200), initialize=150)  # For Distillation Column Separation, initialize with a value

model.all_fAir = Var(bounds=(12000, 15000))  # For Cryogenic Nitrogen
model.all_Plpc = Var(bounds=(0.01, 1.36))  # For Cryogenic Nitrogen
model.P_membrane = Var(bounds=(5, 15))  # For Membrane Nitrogen
model.Fair_membrane = Var(bounds=(2000, 2487.5))  # For Membrane Nitrogen

model.FH2O_dist = Var(bounds=(100, 200))  # For Distillation Column Separation

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [5]:
# Constraints for binary variables to ensure one option is selected per process section
model.Hydrogen_constraint = Constraint(expr=model.SMR + model.WE == 1)
model.Nitrogen_constraint = Constraint(expr=model.CAS + model.MAS == 1)
model.Reactor_constraint = Constraint(expr=model.HB + model.MW == 1)
model.Separation_constraint = Constraint(expr=model.Dist + model.Mem == 1)

# A simple initial objective to maximize the sum of all decision variables (just to initialize)
model.objective = Objective(expr=model.FNG + model.FH2O + model.Tsr + model.Psr + model.T_water + model.N_water +
                                         model.all_fAir + model.all_Plpc + model.P_membrane + model.Fair_membrane + model.FH2O_dist,
                                    sense=minimize)
# Solver setup
solver = SolverFactory('SCIP')

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Manual scaling function
def scale_value(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Iteration Loop to Optimize
max_iterations = 50
tolerance = 1e-4
previous_objective = None

for iteration in range(max_iterations):
    print(f"Iteration {iteration+1}:")

    # Step 1: Solve the Pyomo model (initial run with a simple objective)
    solver.solve(model)

    # Step 2: Extract optimized decision variables
    optimized_FNG = value(model.FNG)
    print(f"FNG: {optimized_FNG}")
    optimized_FH2O = value(model.FH2O)
    print(f"FH2O: {optimized_FH2O}")
    optimized_Tsr = value(model.Tsr)
    print(f"Tsr: {optimized_Tsr}")
    optimized_Psr = value(model.Psr)
    print(f"Psr: {optimized_Psr}")
    optimized_fAir = value(model.all_fAir)
    optimized_Plpc = value(model.all_Plpc)
    optimized_P_membrane = value(model.P_membrane)
    optimized_Fair_membrane = value(model.Fair_membrane)
    optimized_FH2O_dist = value(model.FH2O_dist)
    optimized_T = value(model.T_water)
    optimized_N = value(model.N_water)

    # Initialize all variables
    predicted_capex_hydrogen = 0
    opex_raw_SMR = 0
    predicted_opex_electricity_hydrogen = 0
    predicted_hydrogen_flowrate = 0
    opex_raw_WE = 0
    predicted_capex_nitrogen = 0
    predicted_opex_electricity_nitrogen = 0
    predicted_nitrogen_flowrate = 0
    ammonia_flowrate = 0
    predicted_capex_HB = 0
    predicted_opex_electricity_HB = 0
    predicted_capex_MW = 0
    predicted_opex_electricity_MW = 0
    ammonia_production = 0
    predicted_capex_dist = 0
    predicted_opex_electricity_dist = 0
    predicted_capex_mem = 0
    predicted_opex_electricity_mem = 0
    HB_cooling_water = 0
    CAS_steam = 0
    SMR_cooling_water_steam = 0
    dist_cooling_water_steam_refrigerant = 0

    # Use Keras models to get capex and opex predictions
    if value(model.SMR) == 1:
        # Scale values using manual scaling
        scaled_FNG = scale_value(optimized_FNG, 80, 120)
        scaled_FH2O = scale_value(optimized_FH2O, 255, 375)
        scaled_Tsr = scale_value(optimized_Tsr, 900, 1100)
        scaled_Psr = scale_value(optimized_Psr, 29.6, 46.8)
        
        # Create single input array
        input_data = np.array([[scaled_FNG, scaled_FH2O, scaled_Tsr, scaled_Psr]])
        predicted_capex_hydrogen = EA_SMR_Model.predict(input_data)[0][0]
        
        opex_raw_SMR = optimized_FNG * 24 * 330 * 890000 * 3.66 / 10**6
        predicted_opex_electricity_hydrogen = SMR_ne_model.predict([[optimized_FNG, optimized_FH2O, optimized_Tsr, optimized_Psr]])[0] * 4.2 * 0.1638 * 24 * 3600 * 330/(3.6*10**6)
        predicted_hydrogen_flowrate = SMR_fh2_model.predict([[optimized_FNG, optimized_FH2O, optimized_Tsr, optimized_Psr]])[0]
        SMR_cooling_water_steam = 136 * 24 * 330 * 800.8/585.7
        print(f"SMR CAPEX: {predicted_capex_hydrogen}")

    elif value(model.WE) == 1:
        predicted_capex_hydrogen = 26000000
        predicted_opex_electricity_hydrogen = (WE_NE_model.predict([[value(model.T_water), value(model.N_water)]])[0]) * 10 * 0.1638 * 24 * 3600 * 330/(3.6*10**6)
        predicted_hydrogen_flowrate = WE_H2_model.predict([[value(model.T_water), value(model.N_water)]])[0]
        opex_raw_WE = predicted_hydrogen_flowrate * 24 * 330 * 410.75

    # Similar predictions for nitrogen
    if value(model.CAS) == 1:
        # Scale values using manual scaling
        scaled_fAir = scale_value(optimized_fAir, 12000, 15000)
        scaled_Plpc = scale_value(optimized_Plpc, 0.01, 1.36)
        
        # Create single input array
        input_data = np.array([[scaled_fAir, scaled_Plpc]])
        predicted_capex_nitrogen = EA_CAS_Model.predict(input_data)[0][0] / 15
        
        predicted_opex_electricity_nitrogen = (CASU_ne_model.predict([[value(model.all_fAir), value(model.all_Plpc)]])[0]) * 0.1638 * 24 * 3600 * 330/(15*3.6*10**6)
        predicted_nitrogen_flowrate = (fn2_fair_model.predict([[value(model.all_fAir), value(model.all_Plpc)]])[0])/15
        CAS_steam = 445 * 24 * 330 * 800.8/585.7
        print(f"CAS CAPEX: {predicted_capex_nitrogen}")

    elif value(model.MAS) == 1:
        # Scale values using manual scaling
        scaled_P_membrane = scale_value(optimized_P_membrane, 5, 15)
        scaled_Fair_membrane = scale_value(optimized_Fair_membrane, 2000, 2487.5)
        
        # Create single input array
        input_data = np.array([[scaled_P_membrane, scaled_Fair_membrane]])
        predicted_capex_nitrogen = EA_MAS_Model.predict(input_data)[0][0]
        
        predicted_opex_electricity_nitrogen = MASU_ne_model.predict([[value(model.P_membrane), value(model.Fair_membrane)]])[0] * 1000 * 0.1638 * 24 * 3600 * 330/(3.6*10**6)
        predicted_nitrogen_flowrate = Fn2_P_model.predict([[value(model.P_membrane), value(model.Fair_membrane)]])[0]
        print(f"MAS CAPEX: {predicted_capex_nitrogen}")

    # Ammonia production from reactors
    # Haber Bosch reactor
    if value(model.HB) == 1:
        ammonia_flowrate = Haber_f_model.predict(np.array([[predicted_nitrogen_flowrate[0], predicted_hydrogen_flowrate[0]]]))[0]
        
        # Scale values using manual scaling (adjust bounds as needed)
        scaled_nitrogen = scale_value(predicted_nitrogen_flowrate[0], 0, 1000)
        scaled_hydrogen = scale_value(predicted_hydrogen_flowrate[0], 0, 2000)
        
        # Create single input array
        input_data = np.array([[scaled_nitrogen, scaled_hydrogen]])
        predicted_capex_HB = EA_HB_Model.predict(input_data)[0][0]
        
        predicted_opex_electricity_HB = HB_ne_model.predict(np.array([[predicted_nitrogen_flowrate[0], predicted_hydrogen_flowrate[0]]]))[0][0] * 4.2 * 0.1638 * 24 * 3600 * 330/(3.6*10**6)
        HB_cooling_water = 2 * 24 * 330 * 800.8/585.7
        print(f"HB CAPEX: {predicted_capex_HB}")

    elif value(model.MW) == 1:
        # Microwave reactor
        Vol_n2 = predicted_nitrogen_flowrate * (0.0821*593.15/(1.19*60*5.44))  # ml/h.gcat
        volume_n2 = Vol_n2.reshape(-1,1)

        vol_h2 = predicted_hydrogen_flowrate * (0.0821*593.15/(1.19*60*5.44))  # ml/h.gcat
        
        if isinstance(vol_h2, float):
            vol_h2 = np.array([vol_h2])
        volume_h2 = vol_h2.reshape(-1,1)

        # Calculate the total inlet gas volume
        F = volume_h2 + volume_n2

        # Calculate the gas ratio
        Ratio = volume_h2/volume_n2
        Ratio = Ratio.reshape(-1,1)

        ammonia_conc_MW = (my_model7.predict([[320, 80, Ratio[0][0], F[0][0]]])[0])/100

        x_solution = ammonia_conc_MW*(predicted_nitrogen_flowrate+predicted_hydrogen_flowrate)/(2+2*ammonia_conc_MW)

        molar_n2_mw = (predicted_nitrogen_flowrate - x_solution)*10
        molar_h2_mw = (predicted_hydrogen_flowrate - 3*x_solution)*10
        ammonia_flowrate = 2*x_solution*10

        power_reactor = 340*212766
        predicted_capex_MW = 212766*60000
        predicted_opex_electricity_MW = 340 * 212766 * 0.1638 * 24 * 3600 * 330/(3.6*10**6)

    # Separation
    if value(model.Dist) == 1:
        ammonia_production = dist_column_f_model.predict(np.array([[ammonia_flowrate[0], value(model.FH2O_dist)]]))[0]
        
        # Scale values using manual scaling (adjust bounds as needed)
        scaled_ammonia = scale_value(ammonia_flowrate[0], 0, 500)
        scaled_FH2O_dist = scale_value(value(model.FH2O_dist), 100, 200)
        
        # Create single input array
        input_data = np.array([[scaled_ammonia, scaled_FH2O_dist]])
        predicted_capex_dist = EA_Dist_Model.predict(input_data)[0][0]
        
        dist_cooling_water_steam_refrigerant = 184 * 24 * 330 * 800.8/585.7
        predicted_opex_electricity_dist = dist_ne_model.predict(np.array([[ammonia_flowrate[0], value(model.FH2O_dist)]]))[0] * 4.2 * 0.1638 * 24 * 3600 * 330/(3.6*10**6)
        print(f"Distillation CAPEX: {predicted_capex_dist}")
        
    elif value(model.Mem) == 1:
        ammonia_production = 2.8e-8*100*0.3*10**6*ammonia_flowrate
        predicted_capex_mem = 20000
        predicted_opex_electricity_mem = 0

    # Total expenses - make sure it's a scalar value
    total_expenses = float(
           HB_cooling_water + CAS_steam + SMR_cooling_water_steam + 
           predicted_capex_hydrogen + predicted_capex_nitrogen + 
           predicted_capex_HB + predicted_capex_MW + predicted_capex_dist + 
           predicted_capex_mem + predicted_opex_electricity_hydrogen + 
           predicted_opex_electricity_nitrogen + predicted_opex_electricity_HB + 
           predicted_opex_electricity_MW + predicted_opex_electricity_dist + 
           predicted_opex_electricity_mem + opex_raw_SMR + opex_raw_WE + 
           dist_cooling_water_steam_refrigerant)

    # Update objective - convert to float for Pyomo
    model.objective.expr = total_expenses

    # Step 4: Check for convergence
    current_objective = value(model.objective)
    if previous_objective is not None and abs(current_objective - previous_objective) < tolerance:
        print(f"Converged after {iteration+1} iterations.")
        break

    previous_objective = current_objective

# Final optimized results
print("Optimized hydrogen production route:", "SMR" if value(model.SMR) == 1 else "Water Electrolysis")
print("Optimized nitrogen production route:", "Cryogenic" if value(model.CAS) == 1 else "Membrane")
print("Optimized reactor:", "Haber-Bosch" if value(model.HB) == 1 else "Microwave")
print("Optimized separation:", "Distillation" if value(model.Dist) == 1 else "Membrane")
print("Optimal ammonia production (kmol/h):", ammonia_production)
print("Total expenses in USD:", total_expenses)

Iteration 1:
FNG: 80.0
FH2O: 255.0
Tsr: 900.0
Psr: 29.6
1/1 [==============================] - 0s 20ms/step
SMR CAPEX: 1503950.25
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 23ms/step
CAS CAPEX: 264385.2
1/1 [==============================] - 0s 17ms/step
HB CAPEX: 34054.88671875
1/1 [==============================] - 0s 17ms/step
Distillation CAPEX: 455038.8125
Iteration 2:


/var/folders/m_/xfgqb3bx783_n7rkxqtvk2xc0000gn/T/ipykernel_15237/2402941364.py:181: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  total_expenses = float(


FNG: 80.0
FH2O: 255.0
Tsr: 900.0
Psr: 29.6
1/1 [==============================] - 0s 11ms/step
SMR CAPEX: 1503950.25
1/1 [==============================] - 0s 10ms/step
CAS CAPEX: 264385.2
1/1 [==============================] - 0s 6ms/step
HB CAPEX: 34054.88671875
1/1 [==============================] - 0s 5ms/step
Distillation CAPEX: 455038.8125
Converged after 2 iterations.
Optimized hydrogen production route: SMR
Optimized nitrogen production route: Cryogenic
Optimized reactor: Haber-Bosch
Optimized separation: Distillation
Optimal ammonia production (kmol/h): [69.91862]
Total expenses in USD: 15764800.0
